In [1]:
import pandas as pd
import numpy as np
import requests
import pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/nicholasjones/miniconda3/envs/GPD3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# ThinkHazard API: demo of a simple Python wrapper
This notebook shows a first step to make a ThinkHazard utility in Python. Aims:
* data scientists can get hazard ratings for a given country or admin unit
* syntax is very concise & easy
* takes 2 minutes to learn

### 1. My old example of accessing APIs in Python
Some code to make REST API requests using _requests_ library and parse the resulting json.

In [58]:
endpoint_url = 'http://api.open-notify.org/astros.json'

In [59]:
response = requests.get(endpoint_url)

In [60]:
response.status_code  # 200 = it worked

200

In [61]:
data = response.json()
data

{'message': 'success',
 'number': 7,
 'people': [{'craft': 'ISS', 'name': 'Sergey Ryzhikov'},
  {'craft': 'ISS', 'name': 'Kate Rubins'},
  {'craft': 'ISS', 'name': 'Sergey Kud-Sverchkov'},
  {'craft': 'ISS', 'name': 'Mike Hopkins'},
  {'craft': 'ISS', 'name': 'Victor Glover'},
  {'craft': 'ISS', 'name': 'Shannon Walker'},
  {'craft': 'ISS', 'name': 'Soichi Noguchi'}]}

In [62]:
people_in_space = data['number']
print("There are currently {} people in space".format(people_in_space))

There are currently 7 people in space


### 2. Same for the ThinkHazard API
Get hazard ratings for Pakistan

In [40]:
endpoint = 'http://thinkhazard.org/en/report/188.json'    # 188 = Pakistan

In [63]:
response = requests.get(endpoint)
response.status_code

200

In [64]:
data=response.json()
data[:3]

[{'hazardtype': {'mnemonic': 'FL', 'hazardtype': 'River flood'},
  'hazardlevel': {'mnemonic': 'HIG', 'title': 'High'}},
 {'hazardtype': {'mnemonic': 'UF', 'hazardtype': 'Urban flood'},
  'hazardlevel': {'mnemonic': 'HIG', 'title': 'High'}},
 {'hazardtype': {'mnemonic': 'CF', 'hazardtype': 'Coastal flood'},
  'hazardlevel': {'mnemonic': 'HIG', 'title': 'High'}}]

In [67]:
print(data[0]['hazardtype']['hazardtype'])  # parsing the json
print(data[0]['hazardlevel']['title'])

River flood
High


In [68]:
hazards = [item['hazardtype']['hazardtype'] for item in data]
levels = [item['hazardlevel']['title'] for item in data]

pd.DataFrame({'Hazard': hazards, 'Level':levels}).head()

,Hazard,Level
0,River flood,High
1,Urban flood,High
2,Coastal flood,High
3,Earthquake,High
4,Landslide,High


### 3. Same through an API wrapper
Make it easier using a class 'ThinkHazard' that connects to the API and knows how to parse the output.
User creates a thinkHazard object:
```th = thinkHazard()```)
The object makes several methods available:
* ```hazard_rankings(country_code)``` takes a country or adm unit code and returns the rankings
* other methods can be added.

In [247]:
class thinkHazard():
    def __init__ (self,endpoint='http://thinkhazard.org/en/'):
        self.endpoint=endpoint
        self.country_code_map = {188:'Pakistan',115:'India',175:'Nepal',23:'Bangladesh',231:'Sri Lanka'} # a full country code mapping would be helpful
        self.country_codes = pd.read_csv('https://raw.githubusercontent.com/GFDRR/thinkhazardmethods/master/source/download/ADM2_TH.csv',sep=';',names = ['adm2_code','adm2_name','adm1_code','adm1_name','adm0_code','country_name'],skiprows=1)

    def hazard_rankings(self,country_code):
        # Get the hazard rankings from TH database (chosen country)
        query = self.endpoint + 'report/{}.json'.format(country_code)
        print("query: ",query)
        data = requests.get(query).json()
        hazards = [item['hazardtype']['hazardtype'] for item in data]
        levels = [item['hazardlevel']['title'] for item in data]

        return(pd.DataFrame({'Hazard': hazards, 'Level':levels}))
    
    def hazard_recommendation(self,country_code,hazard,verbose=True):
        # Get the recommendations from TH database (chosen country and hazard)
        query = self.endpoint + 'report/{}/{}.json'.format(country_code,hazard)
        data = requests.get(query).json()
        return(data['hazard_category']['hazard_type'],
               data['hazard_category']['hazard_level'],
               data['hazard_category']['general_recommendation'])
    
    def query(self,countries,hazard):
    # A query function for one hazard across multiple countries.
        levels = []
        for country in countries:
            df=self.hazard_rankings(country)
            levels.append(df['Level'][df.Hazard == hazard].iloc[0])
        country_names = pd.Series(countries).replace(self.country_code_map)
        return(pd.DataFrame({"Country code":countries,"Country name":country_names,"Hazard level":levels}))
    
    def hazard_category(self,hazard,level):
    # Get the general information for the hazard & severity level, as documented  here:
    # https://gfdrr.github.io/thinkhazardmethods/#complete-report-for-a-division-for-a-hazard-type    
        pass

In [230]:
adm0 = pd.read_csv('https://raw.githubusercontent.com/GFDRR/thinkhazardmethods/master/source/download/ADM0_TH.csv',sep=';',usecols=['ADM0_NAME','ADM0_CODE','ISO3166_a3'])
adm0.columns=['adm0_code','country_name','ISO']
adm0.head(2)

,adm0_code,country_name,ISO
0,1,Afghanistan,AFG
1,3,Albania,ALB


In [231]:
adm1 = pd.read_csv('https://raw.githubusercontent.com/GFDRR/thinkhazardmethods/master/source/download/ADM1_TH.csv',sep=';')
adm1.columns = ['adm1_code','adm1_name','adm0_code','country_name']
adm1.head(2)

,adm1_code,adm1_name,adm0_code,country_name
0,272,Badakhshan,1,Afghanistan
1,273,Badghis,1,Afghanistan


In [233]:
adm2 = pd.read_csv('https://raw.githubusercontent.com/GFDRR/thinkhazardmethods/master/source/download/ADM2_TH.csv',sep=';')
adm2.columns = ['adm2_code','adm2_name','adm1_code','adm1_name','adm0_code','country_name']
adm2.head(2)

,adm2_code,adm2_name,adm1_code,adm1_name,adm0_code,country_name
0,487,Aghdash,147298,Aran,19,Azerbaijan
1,488,Aghdam,147306,Yukhari Garabakh,19,Azerbaijan


In [234]:
adm2.shape

(38229, 6)

In [245]:

th=thinkHazard()

In [246]:
th.adm2

,adm2_code,adm2_name,adm1_code,adm1_name,adm0_code,country_name
0,ADM2_CODE,ADM2_NAME,ADM1_CODE,ADM1_NAME,ADM0_CODE,ADM0_NAME
1,487,Aghdash,147298,Aran,19,Azerbaijan
2,488,Aghdam,147306,Yukhari Garabakh,19,Azerbaijan
3,489,Agdjabedi,147298,Aran,19,Azerbaijan
4,490,Aghstafa,147300,Ganja-Gazakh,19,Azerbaijan
...,...,...,...,...,...,...
38225,191421,Isoko North,2218,Delta,182,Nigeria
38226,191422,Lavun,2231,Niger,182,Nigeria
38227,191423,Bade,2240,Yobe,182,Nigeria
38228,191424,Maru,65710,Zamfara,182,Nigeria


In [216]:
th.hazard_rankings(188) # Pakistan

query:  http://thinkhazard.org/en/report/188.json


,Hazard,Level
0,River flood,High
1,Urban flood,High
2,Coastal flood,High
3,Earthquake,High
4,Landslide,High
5,Tsunami,Medium
6,Volcano,Low
7,Cyclone,High
8,Water scarcity,High
9,Extreme heat,High


In [202]:
th.hazard_rankings(203) # Romania

query:  http://thinkhazard.org/en/report/203.json


,Hazard,Level
0,River flood,High
1,Urban flood,High
2,Coastal flood,No Data
3,Earthquake,Medium
4,Landslide,High
5,Tsunami,No Data
6,Volcano,No Data
7,Cyclone,No Data
8,Water scarcity,Medium
9,Extreme heat,Medium


In [203]:
th.hazard_recommendation(188,'FL')  # flood in Pakistan

('River flood',
 'High',
 "In the area you have selected name of location river flood hazard is classified as **high** based on modeled flood information currently available to this tool. This means that potentially damaging and life-threatening river floods are expected to occur at least once in the next 10 years. **Project planning decisions, project design, and construction methods must take into account the level of river flood hazard**. Surface flood hazard in urban and rural areas is not included in this hazard classification, and may also be possible in this location. Please see 'Urban Flood' for consideration of urban surface and river flooding.The following is a list of recommendations that could be followed in different phases of the project to help reduce the risk to your project. Please note that these recommendations are generic and not project-specific.\r\n")

In [205]:
th.query([188,115,175,23,231],'Earthquake') # Earthquake in South Asian countries

query:  http://thinkhazard.org/en/report/188.json
query:  http://thinkhazard.org/en/report/115.json
query:  http://thinkhazard.org/en/report/175.json
query:  http://thinkhazard.org/en/report/23.json
query:  http://thinkhazard.org/en/report/231.json


,Country code,Country name,Hazard level
0,188,Pakistan,High
1,115,India,High
2,175,Nepal,Medium
3,23,Bangladesh,Medium
4,231,Sri Lanka,Very low
